In [35]:
# Capital Spending Project: Joining on CPDB
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import re
import datetime
import matplotlib.font_manager
import time

# Capital Spending Project: Joining on CPDB

In [36]:
# --- checkbook nyc data
file_names = ['../data/checkbooknyc/Citywide-Agencies/spending_29351_05312023_040004_part_0.csv', 
'../data/checkbooknyc/Citywide-Agencies/spending_29351_05312023_040004_part_1.csv',
'../data/checkbooknyc/Citywide-Agencies/spending_29351_05312023_040004_part_2.csv',
'../data/checkbooknyc/Citywide-Agencies/spending_29351_05312023_040004_part_3.csv']
df0 = pd.read_csv(file_names[0])
df1 = pd.read_csv(file_names[1])
df2 = pd.read_csv(file_names[2])
df3 = pd.read_csv(file_names[3])
frames = [df0, df1, df2, df3]
df = pd.concat(frames)

In [37]:
df.head()

,Agency,Associated Prime Vendor,Budget Code,Capital Project,Check Amount,Contract ID,Contract Purpose,Department,Document ID,Emerging Business,Expense Category,Fiscal year,Industry,Issue Date,M/WBE Category,Payee Name,Spending Category,Sub Contract Reference ID,Sub Vendor,Woman Owned Business
0,Transit Authority,NaN,"BUSB (MTA, NYCT: PURCHASE 126 HYBRID BUSES)",998CAP2024 005,99999999.95,CT199820231408687,Bus Purch Track Replace Switch Replace Rail Su...,400-998-169,20230231715-1-DSB-EFT,No,CAPITAL PURCHASED EQUIPMENT,2023,Standardized Services,2022-12-21,Individuals and Others,METROPOLITAN TRANSPORTATION AUTHORITY,Capital Contracts,NaN,No,No
1,Transit Authority,NaN,"SAPS (MTA / NYCT: SUBWAY ACTION PLAN, SIGNAL A)",998CAPSAP 003,99999999.59,CT199820191409664,Subway Action Plan Signal Improvement & Modern...,400-998-169,20190025861-1-015-MD1,No,IOTB CONSTRUCTION,2019,Standardized Services,2018-10-30,Individuals and Others,METROPOLITAN TRANSPORTATION AUTHORITY,Capital Contracts,NaN,No,No
2,Department of Education,NaN,E706 (IMPLEMENTATION OF THE SEVENTH FIVE-YEAR),040SCA22 030,99999999.00,PON1040SCA20-24MC,NaN,IMPLEMENTATION OF THE SEVENTH FIVE-YEAR,20220448629-1-DSB-EFT,No,ADMINISTRATIVE EXPENSES,2022,NaN,2022-05-27,Individuals and Others,NYC SCHOOL CONSTRUCTION AUTHORITY,Capital Contracts,NaN,No,No
3,Department of Education,NaN,E706 (IMPLEMENTATION OF THE SEVENTH FIVE-YEAR),040SCA21 030,99999999.00,PON1040SCA20-24MC,NaN,IMPLEMENTATION OF THE SEVENTH FIVE-YEAR,20220448626-1-DSB-EFT,No,CONSTRUCTION-BUILDINGS,2022,NaN,2022-05-27,Individuals and Others,NYC SCHOOL CONSTRUCTION AUTHORITY,Capital Contracts,NaN,No,No
4,Department of Education,NaN,E706 (IMPLEMENTATION OF THE SEVENTH FIVE-YEAR),040SCA21 030,99999999.00,PON1040SCA20-24MC,NaN,IMPLEMENTATION OF THE SEVENTH FIVE-YEAR,20220354644-1-DSB-EFT,No,CONSTRUCTION-BUILDINGS,2022,NaN,2022-03-25,Individuals and Others,NYC SCHOOL CONSTRUCTION AUTHORITY,Capital Contracts,NaN,No,No


In [38]:
cols = df.columns

cols_dict = {}
for col in cols:
    if col == 'Check Amount':
        cols_dict[col] = 'sum'
    else:
        cols_dict[col] = 'first' # keep first value for all other cols

In [39]:
print(cols_dict)

projects_df = df.groupby(['Capital Project', 'Fiscal year'], as_index=False).agg(cols_dict)

{'Agency': 'first', 'Associated Prime Vendor': 'first', 'Budget Code': 'first', 'Capital Project': 'first', 'Check Amount': 'sum', 'Contract ID': 'first', 'Contract Purpose': 'first', 'Department': 'first', 'Document ID': 'first', 'Emerging Business': 'first', 'Expense Category': 'first', 'Fiscal year': 'first', 'Industry': 'first', 'Issue Date': 'first', 'M/WBE Category': 'first', 'Payee Name': 'first', 'Spending Category': 'first', 'Sub Contract Reference ID': 'first', 'Sub Vendor': 'first', 'Woman Owned Business': 'first'}


In [40]:
projects_df.drop('Issue Date', axis=1, inplace=True)
print(projects_df.shape)
projects_df.head(5)

(187900, 19)


,Agency,Associated Prime Vendor,Budget Code,Capital Project,Check Amount,Contract ID,Contract Purpose,Department,Document ID,Emerging Business,Expense Category,Fiscal year,Industry,M/WBE Category,Payee Name,Spending Category,Sub Contract Reference ID,Sub Vendor,Woman Owned Business
0,Department of Citywide Administrative Services,NaN,None,002FINC01 001,2224671.23,None,None,400-856-714,20100003911-1-DSB-EFT,No,OTPS HOLDING CODE,2010,None,Non-M/WBE,SIDLEY AUSTIN LLP,Capital Contracts,NaN,No,No
1,Department of Citywide Administrative Services,NaN,DUMY (INITIAL COI AUTHORIZATION; FY 2004C),002FINC01 001,3683303.41,None,None,400-856-714,20110184357-1-DSB-AD,No,OTPS HOLDING CODE,2011,None,Non-M/WBE,SIDLEY AUSTIN LLP,Capital Contracts,NaN,No,No
2,Department of Citywide Administrative Services,NaN,DUMY (INITIAL COI AUTHORIZATION; FY 2004C),002FINC01 001,4768011.14,None,None,400-856-714,20120353868-1-DSB-EFT,No,OTPS HOLDING CODE,2012,None,Non-M/WBE,SIDLEY AUSTIN LLP,Capital Contracts,NaN,No,No
3,Department of Citywide Administrative Services,NaN,DUMY (INITIAL COI AUTHORIZATION; FY 2004C),002FINC01 001,3898056.72,None,None,400-856-714,20130171443-1-DSB-EFT,No,OTPS HOLDING CODE,2013,None,Non-M/WBE,SIDLEY AUSTIN LLP,Capital Contracts,NaN,No,No
4,Department of Citywide Administrative Services,NaN,DUMY (INITIAL COI AUTHORIZATION; FY 2004C),002FINC01 001,5337676.18,None,None,400-856-714,20140042557-1-DSB-AD,No,OTPS HOLDING CODE,2014,None,Non-M/WBE,SIDLEY AUSTIN (NY) LLP,Capital Contracts,NaN,No,No
